In [264]:
import torch
#clear memory better
with torch.no_grad():
    torch.cuda.empty_cache()
import numpy as np
import util.npose_util as nu
import os
import pathlib
import dgl
from dgl import backend as F
import torch_geometric
from torch.utils.data import random_split, DataLoader, Dataset
from typing import Dict
from torch import Tensor
from dgl import DGLGraph
from torch import nn
from chemical import cos_ideal_NCAC #from RoseTTAFold2
from torch import einsum
import time
torch.cuda.is_available()

True

In [265]:
import se3_diffuse.utils as du

In [266]:
from data_rigid_diffuser import so3_diffuser
from data_rigid_diffuser import r3_diffuser
from data_rigid_diffuser import oneHot_diffuser
from scipy.spatial.transform import Rotation
from data_rigid_diffuser import rigid_utils as ru
import yaml
from data_rigid_diffuser.diffuser import FrameDiffNoise
from gudiff_model import Data_Graph
from gudiff_model.Data_Graph import build_npose_from_coords, dump_coord_pdb, define_graph_edges, make_pe_encoding
from gudiff_model.Data_Graph import Helix4_Dataset, Make_KNN_MP_Graphs
from gudiff_model.Data_Graph_Null import  Make_nullKNN_MP_Graphs


In [267]:
from se3_transformer.model.basis import get_basis, update_basis_with_fused
from se3_transformer.model.transformer import Sequential, SE3Transformer
from se3_transformer.model.transformer_topk import SE3Transformer_topK
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from se3_transformer.model.layers.attentiontopK import AttentionBlockSE3
from se3_transformer.model.layers.linear import LinearSE3
from se3_transformer.model.layers.convolution import ConvSE3, ConvSE3FuseLevel
from se3_transformer.model.layers.norm import NormSE3
from se3_transformer.model.layers.pooling import GPooling, Latent_Unpool, Unpool_Layer
from se3_transformer.runtime.utils import str2bool, to_cuda
from se3_transformer.model.fiber import Fiber
from se3_transformer.model.transformer import get_populated_edge_features

In [268]:
from se3_transformer.model.FAPE_Loss import FAPE_loss_null, FAPE_loss_real, FAPE_loss
from se3_transformer.model.FAPE_Loss import get_t

In [269]:
#indices for, unsure if needed
CA = Data_Graph.CA
N = Data_Graph.N
C = Data_Graph.C

# #find better way to incorporate coord_scale

#needed
N_CA_dist = (Data_Graph.N_CA_dist/10.).to('cuda')
C_CA_dist = (Data_Graph.C_CA_dist/10.).to('cuda')



In [270]:
# data_path_str  = 'data/h4_ca_coords.npz'
# test_limit = 1028
# rr = np.load(data_path_str)
# ca_coords = [rr[f] for f in rr.files][0][:test_limit,:,:3]
# ca_coords.shape

# getting N-Ca, Ca-C vectors to add as typeI features
#apa = apart helices for val/train split
#tog = together helices for val/train split

#mode for tablet
apa_path_str  = 'data_npose/h4_apa_coords.npz'
tog_path_str  = 'data_npose/h4_tog_coords.npz'

#grab the first 3 atoms which are N,CA,C
test_limit = 2048
rr = np.load(apa_path_str)
coords_apa = [rr[f] for f in rr.files][0][:test_limit,:]

rr = np.load(tog_path_str)
coords_tog = [rr[f] for f in rr.files][0][:test_limit,:]

In [271]:
# B = 16
# L=65
# limit = 1028
# h4_trainData = Helix4_Dataset(coords_tog[:limit])
# h4_valData = Helix4_Dataset(coords_apa[:limit])
# train_dL = DataLoader(h4_trainData, batch_size=B, shuffle=True, drop_last=True)
# val_dL   = DataLoader(h4_valData, batch_size=B, shuffle=True, drop_last=True)
# testiter = iter(train_dL)
# bb_dict = next(testiter)



In [272]:
from data_rigid_diffuser import diffuser

In [273]:
#I think I adjusted inputs and outputs,
#sigmoid for nodes features for real/null pred?
#need loss function for real/null nodes
#need to get function to just pull real nodes for viewing?

In [274]:
#helper functions for Diffusion_Graphical_UNet_Model

def define_poolGraph(n_nodes, batch_size, cast_type=torch.float32, cuda_out=True ):
    
    v1,v2,edge_data, ind = define_graph_edges(n_nodes)
    #pe = make_pe_encoding(n_nodes=n_nodes)#pe e
    
    graphList = []
    
    for i in range(batch_size):
        
        g = dgl.graph((v1,v2))
        g.edata['con'] = edge_data.type(cast_type).reshape((-1,1))
        g.ndata['pos'] = torch.zeros((n_nodes,3),dtype=torch.float32)

        graphList.append(g)
        
    batched_graph = dgl.batch(graphList)
    
    if cuda_out:
        return to_cuda(batched_graph)
    else:
        return batched_graph            
        
def pull_edge_features(graph, edge_feat_dim=1):
    return {'0': graph.edata['con'][:, :edge_feat_dim, None]}

def prep_for_gcn(graph, xyz_pos, edge_feats_input, idx, max_degree=3, comp_grad=True):
    
    src, dst = graph.edges()
    
    new_pos = F.gather_row(xyz_pos, idx)
    rel_pos = F.gather_row(new_pos,dst) - F.gather_row(new_pos,src) 
    
    basis_out = get_basis(rel_pos, max_degree=max_degree,
                                   compute_gradients=comp_grad,
                                   use_pad_trick=False)
    basis_out = update_basis_with_fused(basis_out, max_degree, use_pad_trick=False,
                                            fully_fused=False)
    edge_feats_out = get_populated_edge_features(rel_pos, edge_feats_input)
    return edge_feats_out, basis_out, new_pos    

#--- layer for converting t[0,1] to be more expressive
class GaussianFourierProjection(nn.Module):
    """Gaussian random features for encoding time steps."""  
    #From Yang Song, Tutorial on Score based diffusion models
    def __init__(self, embed_dim, scale=30.):
        super().__init__()
        # Randomly sample weights during initialization. These weights are fixed 
        # during optimization and are not trainable.
        self.W = nn.Parameter(torch.randn(embed_dim // 2) * scale, requires_grad=False)
    def forward(self, x):
        x_proj = x[:, None] * self.W[None, :] * 2 * np.pi
        return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)
    
class GaussianFourierProjection_Linear(nn.Module):
    """Gaussian random features for encoding time steps."""  
    #From Yang Song, Tutorial on Score based diffusion models
    def __init__(self, embed_dim, scale=30., use_deg1 = True):
        super().__init__()
        self.GFP = GaussianFourierProjection(embed_dim, scale=scale)
        self.linear0 = nn.Linear(embed_dim,embed_dim)
        self.use_deg1 = use_deg1
        self.act = nn.SiLU()
        if use_deg1:
            self.linear1 = nn.Linear(embed_dim,1) #create a scalar for multiplication to vector '1'
            #according to to some toronto math notes I believe this retains invariance

    def forward(self, t):
        if self.use_deg1:
            return {'0': self.act(self.linear0(self.GFP(t))), '1':self.act(self.linear1(self.GFP(t)))}
        else:
            return {'0' : self.act(self.linear0(self.GFP(t)))}

In [275]:
##-- Graphical U-net for Denoising
class GraphUNet_Null(torch.nn.Module):
    def __init__(self, 
                 fiber_start = Fiber({0:12, 1:2}),
                 fiber_out = Fiber({0:5,1:2}),
                 k=4,
                 batch_size = 8,
                 stride=4,
                 max_degree=3,
                 channels=32,
                 num_heads = 8,
                 channels_div=4,
                 num_layers = 1,
                 num_layers_ca = 1,
                 edge_feature_dim=1,
                 latent_pool_type = 'avg',
                 t_size = 12,
                 zero_lin=True,
                 use_tdeg1 = True,
                 cuda=True):
        super(GraphUNet_Null, self).__init__()
        
        #fiber_out is 5 for the null nodes, 1deg (2 (3D vector) to update translation/ rotation)
        
        #self.comp_basis_grad = True
        self.cuda = cuda
        
        if cuda:
            self.device='cuda:0'
        else:
            self.device='cpu'
        
        self.max_degree=max_degree #number of 'orbital types' to use for representing the sphere
        self.B = batch_size
        self.k = k #number of midpoints to be chosen to reduce graph size (topK pooling layer)
        self.ts = t_size #number of features to represent t-value
        
        self.use_tdeg1 = use_tdeg1 #apply a t-value to degree (vector) one in the se3transformer
                                    #scalar t put in one dimension , pad zero for other. Perhaps Bad to do.
        self.zero_lin = zero_lin #apply a zero weight value to the linear 
        
        self.embed_t = GaussianFourierProjection_Linear(self.ts, use_deg1=self.use_tdeg1)
        self.t_fiber = Fiber({0:self.ts}) #add for change in fiber with self.concat_t
        
        self.num_layers = 1 #se3 transformers layers for all except the CA graph layer
        self.num_layers_ca = num_layers_ca #layers to use on the CA-alpha graph layer
        self.channels = 32
        self.feat0 = 32 #deg0 number of features
        self.feat1 = 6  #deg1 number of features
        self.channels_div = 4
        self.num_heads = 8 #se3 attention heads
        self.mult = int(stride/2) #multiplier to increase channels as the graph reduces nodes
        self.fiber_edge=Fiber({0:edge_feature_dim})
        self.edge_feat_dim = edge_feature_dim
        
        self.pool_type = latent_pool_type 
        
        self.channels_down_ca = channels #c_alpha interactions by nearest  neighbors onto midpoints 
        #(on the down side of U-Net)
        self.fiber_start =  fiber_start
        self.fiber_hidden_down_ca = Fiber.create(self.max_degree, self.channels_down_ca)
        self.fiber_out_down_ca =Fiber({0: self.feat0, 1: self.feat1})
        
        #concat fiber+t_value, plus one on input fiber, use concat_t method during forward call
        self.down_ca = SE3Transformer(num_layers = self.num_layers_ca,
                        fiber_in=self.fiber_start+self.t_fiber,
                        fiber_hidden= self.fiber_hidden_down_ca, 
                        fiber_out=self.fiber_out_down_ca,
                        num_heads = self.num_heads,
                        channels_div = self.channels_div,
                        fiber_edge=self.fiber_edge,
                        low_memory=True,
                        tensor_cores=False)
        
        self.channels_down_ca2mp = self.channels_down_ca*self.mult
        
        #pool from c_alpha onto midpoints
        self.fiber_in_down_ca2mp     = self.fiber_out_down_ca
        self.fiber_hidden_down_ca2mp = Fiber.create(max_degree, self.channels_down_ca2mp)
        self.fiber_out_down_ca2mp    = Fiber({0: self.feat0*self.mult, 1: self.feat1*self.mult})
        
        #concat_t, plus one on input fiber, run concat_t method on forward call
        self.down_ca2mp = SE3Transformer(num_layers = self.num_layers,
                            fiber_in     = self.fiber_in_down_ca2mp+self.t_fiber,
                            fiber_hidden = self.fiber_hidden_down_ca2mp, 
                            fiber_out    = self.fiber_out_down_ca2mp,
                            num_heads =    self.num_heads,
                            channels_div = self.channels_div,
                            fiber_edge=self. fiber_edge,
                            low_memory=True,
                            tensor_cores=False)
        
        #topK selection of midpoint node graph
        self.fiber_in_mptopk =  self.fiber_out_down_ca2mp
        self.fiber_hidden_down_mp  =self.fiber_hidden_down_ca2mp
        self.fiber_out_down_mp_out =self.fiber_out_down_ca2mp
        self.fiber_out_topkpool=Fiber({0: self.feat0*self.mult*self.mult})
        
        #concat_t, plus one on input fiber, run concat_t method on forward
        self.mp_topk = SE3Transformer_topK(num_layers = self.num_layers,
                                        fiber_in      = self.fiber_in_mptopk+self.t_fiber,
                                        fiber_hidden  = self.fiber_hidden_down_mp, 
                                        fiber_out     = self.fiber_out_down_mp_out ,
                                        fiber_out_topk= self.fiber_out_topkpool,
                                        k             = self.k,
                                        num_heads     = self.num_heads,
                                        channels_div  = self.channels_div,
                                        fiber_edge    =  self.fiber_edge,
                                        low_memory=True,
                                        tensor_cores=False)
        
        self.gsmall = define_poolGraph(self.k, self.B, cast_type=torch.float32, cuda_out=self.cuda)
        self.ef_small = pull_edge_features(self.gsmall, edge_feat_dim=self.edge_feat_dim)
        
        #change to doing convolutions instead of transformer
        self.fiber_in_down_gcn   =  self.fiber_out_topkpool
        self.fiber_out_down_gcn  = Fiber({0: self.feat0*self.mult*self.mult, 1: self.feat1*self.mult})

        self.down_gcn = ConvSE3(fiber_in  = self.fiber_in_down_gcn,
                           fiber_out = self.fiber_out_down_gcn,
                           fiber_edge= self.fiber_edge,
                             self_interaction=True,
                             use_layer_norm=True,
                             max_degree=self.max_degree,
                             fuse_level= ConvSE3FuseLevel.NONE,
                             low_memory= True)
        
        
        self.fiber_in_down_gcnp = self.fiber_out_down_gcn
        #probably rename latent to something more approriate 
        self.latent_size = self.feat0*self.mult*self.mult
        self.fiber_latent = Fiber({0: self.latent_size})

        self.down_gcn2pool = ConvSE3(fiber_in=self.fiber_in_down_gcnp,
                                     fiber_out=self.fiber_latent,
                                     fiber_edge=self.fiber_edge,
                                     self_interaction=True,
                                     use_layer_norm=True,
                                     max_degree=self.max_degree,
                                     fuse_level= ConvSE3FuseLevel.NONE,
                                     low_memory= True)
        
        self.global_pool = GPooling(pool=self.pool_type, feat_type=0)

        self.latent_unpool_layer = Latent_Unpool(fiber_in = self.fiber_latent, fiber_add = self.fiber_out_down_gcn, 
                                            knodes = self.k)

        self.up_gcn = ConvSE3(fiber_in=self.fiber_out_down_gcn,
                             fiber_out=self.fiber_out_down_gcn,
                             fiber_edge=self.fiber_edge,
                             self_interaction=True,
                             use_layer_norm=True,
                             max_degree=self.max_degree,
                             fuse_level= ConvSE3FuseLevel.NONE,
                             low_memory= True)
        
        self.unpool_layer = Unpool_Layer(fiber_in=self.fiber_out_down_gcn, fiber_add=self.fiber_out_down_ca)
        
        self.fiber_in_up_gcn_mp = self.unpool_layer.fiber_out
        self.fiber_hidden_up_mp= self.fiber_hidden_down_ca2mp
        self.fiber_out_up_gcn_mp = self.fiber_out_down_mp_out

        self.up_gcn_mp = SE3Transformer(num_layers = num_layers,
                        fiber_in=self.fiber_in_up_gcn_mp,
                        fiber_hidden= self.fiber_hidden_up_mp, 
                        fiber_out=self.fiber_out_up_gcn_mp,
                        num_heads = self.num_heads,
                        channels_div = self.channels_div,
                        fiber_edge=self.fiber_edge,
                        low_memory=True,
                        tensor_cores=False)
        
        self.unpool_layer_off_mp = Unpool_Layer(fiber_in=self.fiber_out_down_mp_out, fiber_add=self.fiber_out_down_mp_out)

        self.fiber_in_up_off_mp = self.fiber_out_up_gcn_mp
        self.fiber_hidden_up_off_mp = self.fiber_hidden_up_mp
        self.fiber_out_up_off_mp = self.fiber_out_down_ca 
        
        #uses reverse graph to move mp off 
        
        self.up_off_mp = SE3Transformer(num_layers = self.num_layers,
                        fiber_in=self.fiber_in_up_off_mp,
                        fiber_hidden= self.fiber_hidden_up_off_mp, 
                        fiber_out=self.fiber_out_up_off_mp,
                        num_heads = self.num_heads,
                        channels_div = self.channels_div,
                        fiber_edge=self.fiber_edge,
                        low_memory=True,
                        tensor_cores=False)
        
        self.pre_linear = Fiber({0:16,1:36}) 
        #add for 0 for node prediction (real, null), 
        #1 for prediction of 
        
        #concat_t, plus one on input fiber, run concat_t method on forward
        
        self.up_ca = SE3Transformer(num_layers = self.num_layers_ca,
                                    fiber_in=self.fiber_out_down_ca+self.t_fiber,
                                    fiber_hidden= self.fiber_hidden_down_ca, 
                                    fiber_out=self.pre_linear,
                                    num_heads = self.num_heads,
                                    channels_div = self.channels_div,
                                    fiber_edge= self.fiber_edge,
                                    low_memory=True,
                                    tensor_cores=False)
        
        self.fiber_out = fiber_out
        
        self.linear = LinearSE3(fiber_in=self.pre_linear,
                                fiber_out=fiber_out)
        
        #do i need to add activation to zero(linera)
        
        if self.zero_lin:
            self.zero_linear()
        self.act = nn.SiLU()
        self.sig = nn.Sigmoid()
        
    def zero_linear(self):
        """Zero linear weights of degree one only."""
        nn.init.zeros_(self.linear.weights['1'])
        
    def concat_mp_feats(self, ca_feats_in, mp_feats):
        """Concatenate the mp and calpha feats, by debatching batched graph"""
        nf0_c = ca_feats_in['0'].shape[-2]
        nf1_c = ca_feats_in['1'].shape[-2]

        out0_cat_shape = (self.B,self.ca_nodes,-1,1)
        mp0_cat_shape  = (self.B,self.mp_nodes,-1,1)
        out1_cat_shape = (self.B,self.ca_nodes,-1,3)
        mp1_cat_shape  = (self.B,self.mp_nodes,-1,3)

        nf_c = {} #nf_cat
        nf_c['0'] = torch.cat((ca_feats_in['0'].reshape(out0_cat_shape), 
                                 mp_feats['0'].reshape(mp0_cat_shape)[:,-(self.mp_nodes-self.ca_nodes):,:,:]),
                              axis=1).reshape((-1,nf0_c,1))

        nf_c['1'] = torch.cat((ca_feats_in['1'].reshape(out1_cat_shape), 
                                 mp_feats['1'].reshape(mp1_cat_shape)[:,-(self.mp_nodes-self.ca_nodes):,:,:]),
                              axis=1).reshape((-1,nf1_c,3))

        return nf_c
        
    def pull_out_mp_feats(self, ca_mp_feats):
        """Select mp feats selected as the topK nodes"""

        nf0_c = ca_mp_feats['0'].shape[1]
        nf1_c = ca_mp_feats['1'].shape[1]

        nf_mp_ = {}
        #select just mp nodes to move on, the other nodes don't connect but mainting self connections
        nf_mp_['0'] = ca_mp_feats['0'].reshape(self.B,self.mp_nodes,
                                               nf0_c,1)[:,-(self.mp_nodes-self.ca_nodes):,...].reshape((-1,nf0_c,1))
        nf_mp_['1'] = ca_mp_feats['1'].reshape(self.B,self.mp_nodes,
                                               nf1_c,3)[:,-(self.mp_nodes-self.ca_nodes):,...].reshape((-1,nf1_c,3))

        return nf_mp_
    
    def concat_t(self, feats_in, embedded_t, use_deg1=True, cast_type=torch.float):
        """Concatenate T to first position of each tensor. Pad Zeros left for degree 1."""
        feats_out = {}
        key = next(iter(feats_in.keys()))
        shape_tuple = (self.B,-1)+feats_in[key].shape[1:]
        batch_shape = feats_in[key].reshape(shape_tuple).shape
        L = batch_shape[1] #can be ca, ca+mp, mp, k nodes long

        if '0' in feats_in.keys():
            feats_out['0'] = torch.concat((embedded_t['0'][:,None,:,None].repeat(1,L,1,1), 
                                           feats_in['0'].reshape((self.B,L,-1,1))),
                                          axis=2).reshape((self.B*L,-1,1))
        if '1' in feats_in.keys():
            
            if use_deg1:
                feats_out['1'] = torch.multiply(feats_in['1'],
                                                embedded_t['1'][:,None,:,None].repeat(1,L,1,1).reshape((self.B*L,-1,1)))
            else:
                feats_out['1'] = feats_in['1']

        return feats_out
    
    
    def forward(self, feat_dict, batched_t):
        
        
        b_graph, nf, ef = feat_dict['batched_graph'], feat_dict['node_feats'], feat_dict['edge_feats'] 
        b_graph_mp, nf_mp, ef_mp = feat_dict['batched_graph_mp'], feat_dict['node_feats_mp'], feat_dict['edge_feats_mp']
        b_graph_mps, nf_mps, ef_mps =  feat_dict['batched_graph_mpself'], feat_dict['node_feats_mpself'], feat_dict['edge_feats_mpself']
        b_graph_mpRev = feat_dict['batched_graph_mprev'] 

        #assumes equal node numbers in g raphs
        self.ca_nodes = int(b_graph.batch_num_nodes()[0])
        self.mp_nodes = int(b_graph_mp.batch_num_nodes()[0]) #ca+mp nodes number

        #SE3 Attention Transformer, c_alpha
        embed_t = self.embed_t(batched_t)
        t_nf = self.concat_t(nf, embed_t, use_deg1=self.use_tdeg1) #concat_t on
        nf_ca_down_out = self.down_ca(b_graph, t_nf, ef)

        #concatenate on midpoints feats
        
        nf_down_cat_mp = self.concat_mp_feats(nf_ca_down_out, nf_mp)

        #pool from ca onto selected midpoints via SE3 Attention transformer
        #edges from ca to mp only (ca nodes zero after this)
        t_nf_down_cat_mp = self.concat_t(nf_down_cat_mp, embed_t,use_deg1=self.use_tdeg1) #concat_t on
        nf_down_ca2mp_out = self.down_ca2mp(b_graph_mp, t_nf_down_cat_mp, ef_mp)

        #remove ca node feats from tensor 
        nf_mp_out = self.pull_out_mp_feats(nf_down_ca2mp_out)

        t_nf_mp_out = self.concat_t(nf_mp_out, embed_t, use_deg1=self.use_tdeg1) #concat_t on
        node_feats_tk, topk_feats, topk_indx = self.mp_topk(b_graph_mps, t_nf_mp_out, ef_mps)

        #make new basis for small graph of k selected midpoints
        edge_feats_out, basis_out, new_pos = prep_for_gcn(self.gsmall, b_graph_mps.ndata['pos'], self.ef_small,
                                                          topk_indx,
                                                          max_degree=self.max_degree, comp_grad=True)

        down_gcn_out = self.down_gcn(topk_feats, edge_feats_out, self.gsmall,  basis_out)

        down_gcnpool_out = self.down_gcn2pool(down_gcn_out, edge_feats_out, self.gsmall,  basis_out)

        pooled_tensor = self.global_pool(down_gcnpool_out,self.gsmall)
        pooled = {'0':pooled_tensor}
        #----------------------------------------- end of down section
        lat_unp = self.latent_unpool_layer(pooled,down_gcn_out)

        up_gcn_out = self.up_gcn(lat_unp, edge_feats_out, self.gsmall,  basis_out)

        k_to_mp  = self.unpool_layer(up_gcn_out,node_feats_tk,topk_indx)

        up_mp_gcn_out = self.up_gcn_mp(b_graph_mps, k_to_mp, ef_mps)
        
        off_mp_add = {}
        for k,v in up_mp_gcn_out.items():
            off_mp_add[k] = torch.add(up_mp_gcn_out[k],nf_mp_out[k])


        #####triple check from here
        #midpoints node indices for unpool layer
        mp_node_indx = torch.arange(self.ca_nodes,self.mp_nodes, device=self.device)
        mp_idx = mp_node_indx[None,...].repeat_interleave(self.B,0)
        mp_idx =((torch.arange(self.B,device=self.device)*(self.mp_nodes)).reshape((-1,1))+mp_idx).reshape((-1))
        
        #during unpool, keep mp=values and ca=zeros
        zeros_mp_ca = {}
        for k,v in nf_down_cat_mp.items():
            zeros_mp_ca[k] = torch.zeros_like(v, device=self.device)


        unpoff_out = self.unpool_layer_off_mp(off_mp_add, zeros_mp_ca, mp_idx)
        
        out_up_off_mp = self.up_off_mp(b_graph_mpRev, unpoff_out, ef_mp)
        
        #select just ca nodes, mp = zeros from last convolution
        inv_mp_idx= torch.arange(0,self.ca_nodes, device=self.device)
        inv_mp_idx = inv_mp_idx[None,...].repeat_interleave(self.B,0)
        inv_mp_idx =((torch.arange(self.B,device=self.device)*(self.mp_nodes)).reshape((-1,1))
                     +inv_mp_idx).reshape((-1))

        node_final_ca = {}
        for key in out_up_off_mp.keys():
            node_final_ca[key] = torch.add(out_up_off_mp[key][inv_mp_idx,...],nf_ca_down_out[key])

        #return updates 
        t_node_final_ca = self.concat_t(node_final_ca, embed_t, use_deg1=self.use_tdeg1) #concat_t on
        
        output = self.linear(self.up_ca(b_graph, t_node_final_ca, ef))
        output['0'] = self.sig(output['0']) #normalize to zero to one for node calculation
        
        return output

In [276]:
def convert_pV_to_points(dict_in):
    
    CA_fp  = dict_in['bb_firstp']['CA'].reshape(B, L, 3).to(device)
    NC_fp = CA_fp + dict_in['bb_firstp']['N_CA'].reshape(B, L, 3).to(device)
    CC_fp = CA_fp + dict_in['bb_firstp']['C_CA'].reshape(B, L, 3).to(device)
    fp =  torch.cat((NC_fp,CA_fp,CC_fp),dim=2).reshape(B,L,3,3)
    
    CA_lp  = dict_in['bb_firstp']['CA'].reshape(B, L, 3).to(device)
    NC_lp = CA_fp + dict_in['bb_firstp']['N_CA'].reshape(B, L, 3).to(device)
    CC_lp = CA_fp + dict_in['bb_firstp']['C_CA'].reshape(B, L, 3).to(device)
    lp =  torch.cat((NC_lp,CA_lp,CC_lp),dim=2).reshape(B,L,3,3)
    
    return fp, lp
def get_noise_pred_true_null(noised_dict, batched_t, graph_maker, graph_unet):
    
    CA_t  = noised_dict['bb_shifted']['CA'].reshape(B, L, 3).to('cuda')
    NC_t = CA_t + noised_dict['bb_shifted']['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_t = CA_t + noised_dict['bb_shifted']['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)
    
    CA_n  = noised_dict['bb_noised']['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + noised_dict['bb_noised']['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_n = CA_n + noised_dict['bb_noised']['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
    feat_dict = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(feat_dict, batched_t)
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['bb_noised']['N_CA'].reshape(B, L, 3).to('cuda'),
                            noised_dict['bb_noised']['C_CA'].reshape(B, L, 3).to('cuda')),dim=2).reshape(B,L,2,1,3)

    
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:].to('cuda')*N_CA_dist
    CC_p = CA_p + rot_vecs[:,:,1,:].reshape(B, L, 3).to('cuda')*C_CA_dist

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    nf_pred = out['0']
    
    nf_pred = out['0']
    real_nodes_pred = torch.round(nf_pred ).clamp(0,1)
    real_nodes_pred_mask = (real_nodes_pred.squeeze().sum(-1)>1.99).reshape(B,L)
    
    real_nodes_true_mask = noised_dict['real_nodes_mask']
    #place roll here later
#     true = true.to('cpu').numpy()*10
#     noise_xyz = noise_xyz.to('cpu').numpy()*10
#     pred = pred.detach().to('cpu').numpy()*10
    
    
    return true, noise_xyz, pred , real_nodes_pred_mask, real_nodes_true_mask
        
def roll2_continous_true(real_mask_in):
    """Return roll amount to set zero on Nterminal residue for pdb file view"""

    roll_con_out = []
    for i,rmr in enumerate(real_mask_in):
        ep_bool = (rmr^rmr.roll(-1) | rmr^rmr.roll(1)) & rmr
        si = torch.arange(ep_bool.shape[0])[ep_bool]
        #circular if start/end real nodes and we need to roll
        if rmr[0] and rmr[-1]:
            #roll last group across barrier
            roll_con = -si[-1]
        elif not rmr[0]: #move first group to front
            roll_con = -si[0]
        else:
            roll_con=0

        roll_con_out.append(roll_con)

    return roll_con_out
      
def dump_tnp_null(true, noise, pred, t_val, e=0, numOut=1, real_mask=None, pred_mask=None, outdir='output/'):
    
    if numOut>true.shape[0]:
        numOut = true.shape[0]
    
    tnk_dir = f'{outdir}/true_node_mask/'
    pnk_dir = f'{outdir}/pred_node_mask/'
    f_dir = f'{outdir}/full/'
    
    if not os.path.isdir(tnk_dir) and real_mask is not None:
        os.makedirs(tnk_dir)
    if not os.path.isdir(pnk_dir) and pred_mask is not None:
        os.makedirs(pnk_dir)
    if not os.path.isdir(f_dir) and real_mask is not None:
        os.makedirs(f_dir)
    
    if real_mask is not None:
        rc = roll2_continous_true(real_mask)
        for x in range(numOut):
            t_o = true[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            n_o = noise[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            p_o = pred[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            dump_coord_pdb(t_o, fileOut=f'{f_dir}/true_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            dump_coord_pdb(n_o, fileOut=f'{f_dir}/noise_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            dump_coord_pdb(p_o, fileOut=f'{f_dir}/pred_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        
    if pred_mask is not None:
        rc = roll2_continous_true(pred_mask)
        for x,c in enumerate(np.arange(numOut)):
            t_o = true[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            n_o = noise[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            p_o = pred[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            pm = pred_mask[x].roll(int(rc[x]),dims=0)
            dump_coord_pdb(t_o[pm], fileOut=f'{pnk_dir}/true_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            dump_coord_pdb(n_o[pm], fileOut=f'{pnk_dir}/noise_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            dump_coord_pdb(p_o[pm], fileOut=f'{pnk_dir}/pred_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            
    if real_mask is not None:
        rc = roll2_continous_true(real_mask)
        for x,c in enumerate(np.arange(numOut)):
            t_o = true[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            n_o = noise[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            p_o = pred[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            rm = real_mask[x].roll(int(rc[x]),dims=0)
            dump_coord_pdb(t_o[rm], fileOut=f'{pnk_dir}/true_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            dump_coord_pdb(n_o[rm], fileOut=f'{pnk_dir}/noise_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            dump_coord_pdb(p_o[rm], fileOut=f'{pnk_dir}/pred_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            
#     if real_mask is not None:
#         rc = roll2_continous_true(real_mask)
#         for x in range(numOut):
#             dump_coord_pdb(true[x][real_mask[x]], fileOut=f'{tnk_dir}/true_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
#             dump_coord_pdb(noise[x][real_mask[x]], fileOut=f'{tnk_dir}/noise_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
#             dump_coord_pdb(pred[x][real_mask[x]], fileOut=f'{tnk_dir}/pred_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
    
    

In [277]:
def convert_pV_to_points(dict_in,device='cuda'):
    
    CA_fp  = dict_in['bb_firstp']['CA'].to(device)
    NC_fp = CA_fp + dict_in['bb_firstp']['N_CA'].to(device)
    CC_fp = CA_fp +dict_in['bb_firstp']['C_CA'].to(device)
    fp =  torch.cat((NC_fp,CA_fp,CC_fp),dim=2).reshape(B,1,3,3)
    
    CA_lp  = dict_in['bb_lastp']['CA'].to(device)
    NC_lp = CA_lp + dict_in['bb_lastp']['N_CA'].to(device)
    CC_lp = CA_lp + dict_in['bb_lastp']['C_CA'].to(device)
    lp =  torch.cat((NC_lp,CA_lp,CC_lp),dim=2).reshape(B,1,3,3)
    
    return fp, lp
    

In [278]:


def model_step_null(noised_dict, batched_t, graph_maker, graph_unet, train=True):
    #prep coordinates for output display from and comparison via FAPE
    
    CA_t  = noised_dict['bb_shifted']['CA'].reshape(B, L, 3).to(device)
    NC_t = CA_t + noised_dict['bb_shifted']['N_CA'].reshape(B, L, 3).to(device)#not mult by bond distance, seems to help?
    CC_t = CA_t + noised_dict['bb_shifted']['C_CA'].reshape(B, L, 3).to(device)#not mult 
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)

    CA_n  = noised_dict['bb_noised']['CA'].reshape(B, L, 3).to(device)
    NC_n = CA_n + noised_dict['bb_noised']['N_CA'].reshape(B, L, 3).to(device)
    CC_n = CA_n + noised_dict['bb_noised']['C_CA'].reshape(B, L, 3).to(device)
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)

    #prepare graphs
    feat_dict = graph_maker.prep_for_network(noised_dict, cuda=True)
    out =graph_unet(feat_dict,batched_t)
    
    
    #FAPE Loss for the prediction
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation , convert from x,y,z (Quat) to rotate input vectors
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['bb_noised']['N_CA'].reshape(B, L, 3).to(device),
                            noised_dict['bb_noised']['C_CA'].reshape(B, L, 3).to(device)),dim=2).reshape(B,L,2,1,3)
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:]*N_CA_dist #comparable but seems better not have it for true, but have it for pred
    CC_p = CA_p + rot_vecs[:,:,1,:]*C_CA_dist #maybe this helep prevent 

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    #divide loss by real and null nodes
    
    fp, lp  = convert_pV_to_points(noised_dict)

    real_mask = noised_dict['real_nodes_mask'].to('cuda')
    score_scales = noised_dict['score_scales'].to('cuda')

    lr, lr_d = FAPE_loss_real(pred, true, score_scales, real_mask,  d_clamp=10.0, d_clamp_inter=30.0,
                   A=10.0, gamma=score_weights['3D_real'], eps=1e-6)
    ln, ln_d = FAPE_loss_null(pred, fp, lp, real_mask, true, score_scales,  d_clamp=10.0,
                       d_clamp_inter=30.0, A=10.0, gamma=score_weights['3D_null'], eps=1e-6)
    
    structure_loss = lr*score_weights['3D_real']+ln*score_weights['3D_null']
    
    #score for node feats determining whether node is real or fake
    nf_pred = out['0']

    nf_feat_dim = noised_dict['real_nodes_noise'].shape[-1]
    nf_true = torch.ones(noised_dict['real_nodes_mask'].shape+(nf_feat_dim,) + (1,),
                         dtype=torch.float,device=device)

    nf_real_mask_mult = real_mask.unsqueeze(-1).unsqueeze(-1).to(device)
    nf_true = nf_true*nf_real_mask_mult

    nf_pred = nf_pred.reshape(B,-1,nf_feat_dim)
    pred_nf_loss = torch.sum(torch.abs(nf_true.squeeze()-nf_pred),dim=-1) #absolute value loss
    pred_nf_loss = pred_nf_loss.to(device)
    
    ss_scales = to_cuda(noised_dict['score_scales'])[:,None,None]
    pnfloss = (torch.sum((pred_nf_loss*ss_scales/L)))*score_weights['nf_real']
    
    final_loss = structure_loss + pnfloss
    
    
    return final_loss, pnfloss.detach().cpu(), structure_loss.detach().cpu()



In [279]:
device='cuda'

B = 2
L=128
limit = 1028
prot_trainData = Data_Graph.ProteinBB_Dataset(coords_tog[:limit], n_nodes=L,
              n_atoms=5, coord_div=10, cast_type=torch.float32)
train_dL = DataLoader(prot_trainData, batch_size=B, shuffle=True, drop_last=True)
stride=4
mkg = Make_nullKNN_MP_Graphs(KNN=30, mp_stride=stride, n_nodes=L)

score_weights = {}
score_weights['nf_real'] = torch.tensor(0,device=device)
score_weights['3D_real'] = torch.tensor(1.0,device=device)
score_weights['3D_null'] = torch.tensor(1.0,device=device)

config_path='data_rigid_diffuser/base.yaml'
fnd = FrameDiffNoise(config_path)


In [280]:
gunn= GraphUNet_Null(fiber_start = Fiber({0:17, 1:2}),
                     fiber_out = Fiber({0:5,1:2}),
                      k=8,
                      batch_size = B,
                      stride=stride,
                       max_degree=3,
                       channels=32,
                      num_heads = 8,
                      channels_div=4,
                      num_layers = 1,
                     num_layers_ca = 2,
                     edge_feature_dim=1,
                     latent_pool_type = 'avg',
                     t_size = 12,
                    zero_lin=True,
                   use_tdeg1 = False,
                 cuda=True).to('cuda')

opti = torch.optim.Adam(gunn.parameters(), lr=0.001, weight_decay=5e-7)

In [449]:
def FAPE_loss(pred, true, score_scales,  d_clamp=10.0, d_clamp_inter=30.0, A=10.0, gamma=1.0, eps=1e-6):
    '''
    Calculate Backbone FAPE loss from RosettaTTAFold
    https://github.com/uw-ipd/RoseTTAFold2/blob/main/network/loss.py
    Input:
        - pred: predicted coordinates (I, B, L, n_atom, 3)
        - true: true coordinates (B, L, n_atom, 3)
    Output: str loss
    '''
    I = pred.shape[0]
    true = true.unsqueeze(0)
    t_tilde_ij = get_t(true[:,:,:,0], true[:,:,:,1], true[:,:,:,2])
    t_ij = get_t(pred[:,:,:,0], pred[:,:,:,1], pred[:,:,:,2])

    difference = torch.sqrt(torch.square(t_tilde_ij-t_ij).sum(dim=-1) + eps)
    eij_label = difference[-1].clone().detach()

    clamp = torch.zeros_like(difference)

    # intra vs inter#me coded
    clamp[:,True] = d_clamp

    difference = torch.clamp(difference, max=clamp)
    loss = difference / A # (I, B, L, L)
    # calculate masked loss (ignore missing regions when calculate loss)
    loss = (loss[:,True]).sum(dim=-1) / (torch.ones_like(loss).sum()+eps) # (I)
    
    # weighting loss
#     w_loss = torch.pow(torch.full((I,), gamma, device=pred.device), torch.arange(I, device=pred.device))
#     w_loss = torch.flip(w_loss, (0,))
#     w_loss = w_loss / w_loss.sum()
    w_loss = score_scales[None,None,:,None]

    tot_loss = (w_loss * loss).sum()
    
    return tot_loss, loss.detach()
def FAPE_loss_real(pred, true, score_scales, real_mask, 
                   d_clamp=10.0, d_clamp_inter=30.0, A=10.0, gamma=1.0, eps=1e-6):
    '''
    Calculate Backbone FAPE loss from RosettaTTAFold
    https://github.com/uw-ipd/RoseTTAFold2/blob/main/network/loss.py
    Input:
        - pred: predicted coordinates (I, B, L, n_atom, 3)
        - true: true coordinates (B, L, n_atom, 3)
    Output: str loss
    '''
    batch = true.shape[0]
    length = true.shape[1]
    pred = pred.unsqueeze(0) #maybe swap back for consistenccy
    true = true.unsqueeze(0)
    
    t_tilde_ij = get_t(true[:,:,:,0], true[:,:,:,1], true[:,:,:,2])
    t_ij = get_t(pred[:,:,:,0], pred[:,:,:,1], pred[:,:,:,2])
    
    difference = torch.sqrt(torch.square(t_tilde_ij-t_ij).sum(dim=-1) + eps)
    clamp = torch.zeros_like(difference)
    # intra vs inter#me coded
    clamp[:,True] = torch.tensor(d_clamp)
    difference = torch.clamp(difference, max=clamp)
    loss = difference / A # (I, B, L, L)

    # n points, becomes nxn comparisons for FAPE LOSS, need to mask both dimensions
    rm_e = real_mask.repeat((1,length)).reshape(1,batch,length,length)
    rm_et =  torch.transpose(rm_e,2,3)

    loss_norm_masked = (rm_e*loss*rm_et).sum(dim=(2,3))/((rm_e*rm_et).sum(dim=(2,3)))
    w_loss = score_scales[None,:]

    tot_loss = (w_loss * loss_norm_masked).sum()
    
    return tot_loss, loss.detach()

def FAPE_loss_null(pred, first_point, last_point, real_mask, score_scales,  d_clamp=10.0,
                   d_clamp_inter=30.0, A=10.0, gamma=0.1, eps=1e-6):
    '''
    Calculate Backbone FAPE loss from RosettaTTAFold
    https://github.com/uw-ipd/RoseTTAFold2/blob/main/network/loss.py
    Input:
        - pred: predicted coordinates (I, B, L, n_atom, 3)
        - first_point: true coordinate (B, 1, n_atom, 3) N-terminal location
        - last_point: true coordinates (B, 1, n_atom, 3) C-terminal location
        chooses lowest loss of null to c-terminal/ n-terminal since its indeterminant
        gamma set to overweight weight of null
    Output: str loss
    
    
    '''
    first_point = first_point.unsqueeze(0)
    last_point = last_point.unsqueeze(0)

    pred = pred.unsqueeze(0) #maybe swap back for consistenccy
    true = torch.ones_like(pred)
    null_mask = (~real_mask).unsqueeze(0) 

    #add first and last point to characterize FAPE loss to the closest proper null node (endpoints)
    flp_pred = torch.concat((first_point,last_point,pred),2)
    flp_true = torch.concat((first_point,last_point,true),2)

    t_tilde_ij = get_t(flp_true[:,:,:,0], flp_true[:,:,:,1], flp_true[:,:,:,2])
    t_ij = get_t(flp_pred[:,:,:,0], flp_pred[:,:,:,1], flp_pred[:,:,:,2])

    difference = torch.sqrt(torch.square(t_tilde_ij-t_ij).sum(dim=-1) + eps)
    clamp = torch.zeros_like(difference)
    
    # intra vs inter #multimer monomer i think, #coding out
    clamp[:,True] = torch.tensor(d_clamp)
    difference = torch.clamp(difference, max=clamp)
    
    #null nodes, least difference to just two points
    #reduces shape by last dimension
    difference_fp = difference[:,:,2:,0]
    difference_lp = difference[:,:,2:,1]
    nearest_endpoint = difference_lp.clone()
    nearest_endpoint[difference_fp<difference_lp] = difference_fp[difference_fp<difference_lp]
    
    loss = nearest_endpoint / A # (I, B, L)
    (null_mask*loss).sum(dim=(2))/((null_mask).sum(dim=(2)))

    loss_norm_masked= (null_mask*loss).sum(dim=(2))/((null_mask).sum(dim=(2)))
    
    w_loss = score_scales[None,:]
    tot_loss = (w_loss * loss_norm_masked).sum()
    
    return tot_loss, loss.detach()

In [433]:
testiter = iter(train_dL)
bb_dict = next(testiter)

In [460]:
B=2
L=128
t=0.0000001
t_cpu = np.ones((B,))*t
noised_dict = fnd.forward(bb_dict,t_vec=t_cpu)
batched_t = to_cuda(noised_dict['t_vec'])

CA_t  = noised_dict['bb_shifted']['CA'].reshape(B, L, 3)
NC_t = CA_t + noised_dict['bb_shifted']['N_CA'].reshape(B, L, 3)#not mult by bond distance, seems to help?
CC_t = CA_t + noised_dict['bb_shifted']['C_CA'].reshape(B, L, 3)#not mult 
true_save =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)

CA_n  = noised_dict['bb_noised']['CA'].reshape(B, L, 3)
NC_n = CA_n + noised_dict['bb_noised']['N_CA'].reshape(B, L, 3)
CC_n = CA_n + noised_dict['bb_noised']['C_CA'].reshape(B, L, 3)
noise_xyz_save =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)

score_scales = torch.ones_like(noised_dict['score_scales'])
real_mask_save =  noised_dict['real_nodes_mask']

fp_save, lp_save  = convert_pV_to_points(noised_dict,device='cpu')

n=0
ld, ln_d = FAPE_loss(noise_xyz_save[n][real_mask_save[n]].unsqueeze(0).unsqueeze(0),
                     true_save[n][real_mask_save[n]].unsqueeze(0),
                     score_scales[0].unsqueeze(0),  d_clamp=10.0, d_clamp_inter=30.0, A=10.0, gamma=1.0, eps=1e-6)
print(ld)

ld, ln_d = FAPE_loss_null(noise_xyz_save, fp_save, lp_save, real_mask_save,score_scales,  d_clamp=10.0,
                   d_clamp_inter=30.0, A=10.0, gamma=0.1, eps=1e-6)
# ld, ln_d = FAPE_loss(noise_xyz_save.unsqueeze(0), true_save, score_scales,  d_clamp=10.0, 
#                      d_clamp_inter=30.0, A=10.0, gamma=1.0, eps=1e-6)

tensor(0.0179)


In [464]:
d_clamp=10.0,
d_clamp_inter=30.0
A=10.0
gamma=0.1
eps=1e-6

pred = noise_xyz_save.unsqueeze(0) #maybe swap back for consistenccy
true = true_save.unsqueeze(0)
real_mask = real_mask_save[0].unsqueeze(0)
    
t_tilde_ij = get_t(true[:,:,:,0], true[:,:,:,1], true[:,:,:,2])
t_ij = get_t(pred[:,:,:,0], pred[:,:,:,1], pred[:,:,:,2])
    
difference = torch.sqrt(torch.square(t_tilde_ij-t_ij).sum(dim=-1) + eps)
clamp = torch.zeros_like(difference)
# intra vs inter#me coded
clamp[:,True] = torch.tensor(d_clamp)
difference = torch.clamp(difference, max=clamp)
loss = difference / A # (I, B, L, L)

loss = loss[:,0,...].unsqueeze(0)

rm_e = real_mask.repeat((1,L)).reshape(1,1,L,L)
rm_et =  torch.transpose(rm_e,2,3)
print(rm_et.shape)
print((rm_e*loss*rm_et).sum(dim=(2,3)))
(rm_e*loss*rm_et).sum(dim=(2,3))/((rm_e*rm_et).sum(dim=(2,3)))

torch.Size([1, 1, 128, 128])
tensor([[75.5072]])


tensor([[0.0179]])

In [437]:
n=0
ld, ln_d =  FAPE_loss_real(noise_xyz_save[n].unsqueeze(0),
                     true_save[n].unsqueeze(0),
                     score_scales[0].unsqueeze(0), 
                            real_mask[0].unsqueeze(0),
                           d_clamp=10.0, d_clamp_inter=30.0, A=10.0, gamma=1.0, eps=1e-6)

In [438]:
d_clamp=10.0,
d_clamp_inter=30.0
A=10.0
gamma=0.1
eps=1e-6

first_point = fp_save.clone().unsqueeze(0)
last_point = lp_save.clone().unsqueeze(0)
#first_point = lp_save.clone().unsqueeze(0)

pred = noise_xyz_save.unsqueeze(0) #maybe swap back for consistenccy
true = true_save.unsqueeze(0)
null_mask = ~real_mask_save.unsqueeze(0) 

#fp = lp_save.repeat((1,L,1,1)).unsqueeze(0)
print(first_point.shape)
print(pred.shape)

#add first and last point to characterize FAPE loss to the closest proper null node (endpoints)
flp_pred = torch.concat((first_point,last_point,pred),2)
flp_true = torch.concat((first_point,last_point,true),2)


#this doesn't work since all points are the same
t_tilde_ij = get_t(flp_true[:,:,:,0], flp_true[:,:,:,1], flp_true[:,:,:,2])
t_ij = get_t(flp_pred[:,:,:,0], flp_pred[:,:,:,1], flp_pred[:,:,:,2])

difference = torch.sqrt(torch.square(t_tilde_ij-t_ij).sum(dim=-1) + eps)

print('diff',difference.shape)
clamp = torch.zeros_like(difference)
# intra vs inter#me coded
clamp[:,True] = torch.tensor(d_clamp)
difference = torch.clamp(difference, max=clamp)
print(difference.shape)
#null nodes
difference_fp = difference[:,:,2:,0]
difference_lp = difference[:,:,2:,1]

difference_fp
print(difference_lp.shape)
nearest_endpoint = difference_lp.clone()
nearest_endpoint[difference_fp<difference_lp] = difference_fp[difference_fp<difference_lp]
print(nearest_endpoint.shape)
loss = nearest_endpoint / A # (I, B, L, L)
print(loss.sum())
print(loss.shape)
#nm_et =  torch.transpose(nm_e,1,2)
(null_mask*loss).sum(dim=(2))/((null_mask).sum(dim=(2)))



torch.Size([1, 2, 1, 3, 3])
torch.Size([1, 2, 128, 3, 3])
diff torch.Size([1, 2, 130, 130])
torch.Size([1, 2, 130, 130])
torch.Size([1, 2, 128])
torch.Size([1, 2, 128])
tensor(1.9719)
torch.Size([1, 2, 128])


tensor([[0.0001, 0.0001]])

In [470]:
check(bb_dict,t=0.0000001, jp_fape=True)

tensor(0.0148) FAPE_Regular_ALL
tensor(0.0148) FAPE_REAL
tensor(0.0002) FAPE_null


In [471]:
check(bb_dict,t=0.01, jp_fape=True)

tensor(0.0220) FAPE_Regular_ALL
tensor(0.0220) FAPE_REAL
tensor(0.0148) FAPE_null


In [472]:
check(bb_dict,t=0.05, jp_fape=True)

tensor(0.0548) FAPE_Regular_ALL
tensor(0.0548) FAPE_REAL
tensor(0.0544) FAPE_null


In [473]:
check(bb_dict,t=0.1, jp_fape=True)

tensor(0.0927) FAPE_Regular_ALL
tensor(0.0927) FAPE_REAL
tensor(0.1014) FAPE_null


In [474]:
check(bb_dict,t=0.5, jp_fape=True)

tensor(0.2540) FAPE_Regular_ALL
tensor(0.2540) FAPE_REAL
tensor(0.3128) FAPE_null


In [475]:
check(bb_dict,t=1.0, jp_fape=True)

tensor(0.2560) FAPE_Regular_ALL
tensor(0.2560) FAPE_REAL
tensor(0.3526) FAPE_null


In [469]:
def check(bb_dict, t=0.1,n=0, jp_fape=True):
    
    t_cpu = np.ones((B,))*t
    noised_dict = fnd.forward(bb_dict,t_vec=t_cpu)
    batched_t = to_cuda(noised_dict['t_vec'])

    if not jp_fape:
        rnm = noised_dict['real_nodes_mask'][n]
        rr = roll2_continous_true(noised_dict['real_nodes_mask'])
        real_indices = torch.arange(128).roll(int(rr[n]))
        rnm = rnm.roll((int(rr[n])))
        first_real_index = real_indices[rnm][0]
        last_real_index = real_indices[rnm][-1]

        print('first real index',first_real_index)
        print('last_real_index',last_real_index)
        print('real_indices',real_indices)

        fpc1 = noised_dict['bb_firstp']['CA'][n]
        fpc1t = noised_dict['bb_shifted']['CA'][n][first_real_index ]
        print('fp_ca_recorded:   ',fpc1)
        print('fp_cafpc1t_found: ',fpc1t )
        print()
        lpc1 = noised_dict['bb_lastp']['CA'][n]
        lpc1t = noised_dict['bb_shifted']['CA'][n][last_real_index ]
        print('lp_ca_recorded:   ',lpc1)
        print('lp_cafpc1t_found: ',lpc1t )
        print()
        fpc1 = noised_dict['bb_firstp']['N_CA'][n]
        fpc1t = noised_dict['bb_shifted']['N_CA'][first_real_index ]
        print('fp_nca_recorded:   ',fpc1)
        print('fpc1t_nca_found: ',fpc1t )

        print()
        lpc1 = noised_dict['bb_lastp']['N_CA'][n]
        lpc1t = noised_dict['bb_shifted']['N_CA'][last_real_index ]
        print('lp_nca_recorded:   ',lpc1)
        print('lpc1t_nca_found: ',lpc1t )

        fpc1 = noised_dict['bb_firstp']['C_CA'][n]
        fpc1t = noised_dict['bb_shifted']['C_CA'][first_real_index ]
        print('fp_nca_recorded:   ',fpc1)
        print('fpc1t_nca_found: ',fpc1t )

        print()
        lpc1 = noised_dict['bb_lastp']['C_CA'][n]
        lpc1t = noised_dict['bb_shifted']['C_CA'][last_real_index ]
        print('lp_nca_recorded:   ',lpc1)
        print('lpc1t_nca_found: ',lpc1t )
    
    CA_t  = noised_dict['bb_shifted']['CA'].reshape(B, L, 3)
    NC_t = CA_t + noised_dict['bb_shifted']['N_CA'].reshape(B, L, 3)#not mult by bond distance, seems to help?
    CC_t = CA_t + noised_dict['bb_shifted']['C_CA'].reshape(B, L, 3)#not mult 
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)

    CA_n  = noised_dict['bb_noised']['CA'].reshape(B, L, 3)
    NC_n = CA_n + noised_dict['bb_noised']['N_CA'].reshape(B, L, 3)
    CC_n = CA_n + noised_dict['bb_noised']['C_CA'].reshape(B, L, 3)
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
    fp_save, lp_save  = convert_pV_to_points(noised_dict,device='cpu')

    score_scales = torch.ones_like(noised_dict['score_scales'])
    real_mask =  noised_dict['real_nodes_mask']
    ld, ln_d = FAPE_loss(noise_xyz.unsqueeze(0), true,score_scales,  d_clamp=10.0,
                       d_clamp_inter=30.0, A=10.0, gamma=1.0, eps=1e-6)
    print(ld,"FAPE_Regular_ALL")
    
    lr, ln_dr = FAPE_loss_real(noise_xyz, true, score_scales, real_mask, d_clamp=10.0,
                   d_clamp_inter=30.0, A=10.0, gamma=1.0, eps=1e-6)
    print(ld,"FAPE_REAL")
    
    pred, first_point, last_point, real_mask, score_scales
    
    ln, ln_dr = FAPE_loss_null(noise_xyz, fp_save, lp_save, real_mask, score_scales, d_clamp=10.0,
                   d_clamp_inter=30.0, A=10.0, gamma=1.0, eps=1e-6)
    print(ln,"FAPE_null")
    
    